# Graph Algebra with `kglab`

## intro
`kglab` provides tools to access graph data from multiple source to build a `KnowledgeGraph` that can be easily used by data scientists. For a thorough explanation of how to use triples-stored data and how to load this data into `kglab` please see examples in the `examples/` directory. The examples in this directory (`examples/graph_algebra/`) will care to introduce graph algebra capabilities to be used on the graphs the user has loaded. 

## basic load and querying
In particular, once your data is loaded in a `KnowledgeGraph` with something like:

1. Instantiate a graph from a dataset:

In [1]:
# for use in tutorial and development; do not include this `sys.path` change in production:
import sys ; sys.path.insert(0, "../../")
from os.path import dirname
import kglab
import os

namespaces = {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "gorm": "http://example.org/sagas#",
    "rel":  "http://purl.org/vocab/relationship/",
    }

kg = kglab.KnowledgeGraph(
    name = "Happy Vikings KG example for SKOS/OWL inference",
    namespaces=namespaces,
    )

kg.load_rdf(dirname(dirname(os.getcwd())) + "/dat/gorm.ttl")


2. It is possible to create a subgraph by providing a SPARQL query, by defining a "subject" and "object":


In [2]:
query = """SELECT ?subject ?object
WHERE {
    ?subject rdf:type gorm:Viking .
    ?subject gorm:childOf ?object .
}
"""


## define a subgraph
In this case we are looking for the network of parent-child relations among members of Vikings family.

With this query we can define a **subgraph** so to have access to **graph algebra** capabilities: 

In [3]:
from kglab.subg import SubgraphMatrix

subgraph = SubgraphMatrix(kg=kg, sparql=query)


## compute Adjacency matrices
Let's compute the first basic adjacency matrix (usually noted with `A`):

In [4]:
adj_matrix = subgraph.to_adjacency()
adj_matrix

array([[0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])

what happened here is that all the subjects and objects have been turned into integer indices from 0 to number of nodes. So we can see that the entity with index 0 is adjancent (is connected, has a directed edge) to the entity with index 1. This is a directed graph because the relationship `gorm:childOf` goes from child to parent, let's turn this into an undirected graph so to see the relation in a more symmetric way (both the child-parent and parent-child).

In [5]:
undirected_adj_mtx = subgraph.to_undirected()
undirected_adj_mtx

array([[0., 1., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

We can see now the relationship is a generic symmetric "parenthood" relations, not just a child-parent directed relationship.